In [1]:
// Kotlin notebook
import jline.*

GlobalConstants.setVerbose(VerboseLevel.STD);

In [2]:
// Create the M/E_r/1 model using gallery function from GettingStarted
val modelComponents = GettingStarted.gallery_merl1()
val model = modelComponents[0] as Network
val source = modelComponents[1] as Source
val queue = modelComponents[2] as Queue
val sink = modelComponents[3] as Sink
val oclass = modelComponents[4] as OpenClass

In [3]:
// Configure CTMC solver with proper seeding using SolverOptions structure
// Create options object separately and pass to constructor
val options = SolverOptions()
options.cutoff = Matrix.singleton(150.0)  // Create a singleton matrix with value 150
options.seed = 23000
options.verbose = VerboseLevel.SILENT

// Use the constructor that takes SolverOptions directly
val solver = CTMC(model, options)

In [ ]:
// Sample the system aggregate state
val sa = solver.sampleSysAggr(5000)
val ind = model.getNodeIndex(queue)

// Filter events for departures from the queue
val depTimes = mutableListOf<Double>()
for (i in 0 until sa.event.getNumRows()) {
    val eventTime = sa.event.get(i, 0)
    val eventNode = sa.event.get(i, 1).toInt()
    val eventType = sa.event.get(i, 2).toInt()
    
    // EventType.DEP is typically represented as 2 in the event matrix
    if (eventNode == ind && eventType == 2) {
        depTimes.add(eventTime)
    }
}

println("Found ${depTimes.size} departure events from queue")

In [21]:
if (depTimes.size > 1) {
    // Calculate inter-departure times
    val interDepTimes = mutableListOf<Double>()
    for (i in 1 until depTimes.size) {
        interDepTimes.add(depTimes[i] - depTimes[i-1])
    }
    
    // Calculate mean and variance
    val mean = interDepTimes.average()
    val variance = interDepTimes.map { (it - mean).pow(2) }.average()
    
    // Estimated squared coefficient of variation of departures
    val scvDEst = variance / mean.pow(2)
    println("Simulated SCV of departures: $scvDEst")
} else {
    println("Error: Insufficient departure events found")
    println("Total events generated: ${sa.event.getNumRows()}")
}

Simulated SCV of departures: 0.8606197895029165


In [22]:
// Get queue utilization
val util = solver.getAvgUtil()
val utilQueue = util.get(queue.nodeIndex, 0)

// Get average waiting time (excluding service)
val avgWaitTime = solver.getAvgWaitT()
val avgWaitTimeQueue = avgWaitTime.get(queue.nodeIndex)

In [23]:
// Get arrival and service process parameters
val scvA = source.getArrivalProcess(oclass).getSCV()
val svcRate = queue.getServiceProcess(oclass).getRate()
val scvS = queue.getServiceProcess(oclass).getSCV()

// Marshall's exact formula
val scvD = scvA + 2 * utilQueue.pow(2) * scvS - 2 * utilQueue * (1 - utilQueue) * svcRate * avgWaitTimeQueue
println("Theoretical SCV of departures (Marshall's formula): $scvD")

Theoretical SCV of departures (Marshall's formula): 0.8750004532302259


In [24]:
// Calculate relative error between simulated and theoretical SCV
if (depTimes.size > 1) {
    // Recalculate scvDEst to ensure it is in scope
    val interDepTimes = mutableListOf<Double>()
    for (i in 1 until depTimes.size) {
        interDepTimes.add(depTimes[i] - depTimes[i-1])
    }
    val mean = interDepTimes.average()
    val variance = interDepTimes.map { (it - mean).pow(2) }.average()
    val scvDEst = variance / mean.pow(2)
    
    val relativeError = abs(scvDEst - scvD) / scvD * 100
    
    println("\n=== Departure Process Analysis Results ===")
    println("Simulated SCV of departures:   %.6f".format(scvDEst))
    println("Theoretical SCV (Marshall):    %.6f".format(scvD))
    println("Relative error:                %.2f%%".format(relativeError))
} else {
    println("\nCannot calculate relative error - simulation failed")
}


=== Departure Process Analysis Results ===
Simulated SCV of departures:   0.860620
Theoretical SCV (Marshall):    0.875000
Relative error:                1.64%
